# Simulation2

**Mengqi Liu**

**Jun 16, 2023**
___

In [1]:
import numpy as np
import pandas as pd
import math
from itertools import permutations, combinations
import statsmodels.formula.api as smf
import multiprocessing as mp
from tqdm import tqdm
import matplotlib.pyplot as plt
import statsmodels.stats.multitest as ssm

In [8]:
def data_generative1(N=100, s=1):
    np.random.seed(s); Z = np.random.uniform(0, 10, N)

    def fz(Z):
        return np.array([np.exp(math.sin(z) - np.log(4)) for z in Z])
    
    np.random.seed(s); X = np.random.binomial(1, fz(Z))
    np.random.seed(s+100); Y = np.random.binomial(1, fz(Z))
    return X, Y, Z
X, Y, Z = data_generative1(N=100)

In [9]:
modx = smf.ols(formula='X ~ Z', data=pd.DataFrame({'X':X, 'Y':Y, 'Z':Z})).fit()

In [12]:
modx = smf.ols(formula='X ~ Z', data=pd.DataFrame({'X':X, 'Y':Y, 'Z':Z})).fit()
mody = smf.ols(formula='Y ~ Z', data=pd.DataFrame({'X':X, 'Y':Y, 'Z':Z})).fit()
T = np.abs(np.corrcoef(modx.resid, mody.resid)[0, 1])

In [25]:
def discretize(X, M = 10):
    '''Discretize continuous variable X

    Args: 
        M: num of bins
    
    Returns:
        new_X: (np.array) discretized X with values of midpoints in all bins
    '''
    bins = np.linspace(np.min(X), np.max(X), M+1)
    mid_bins = (bins[0:-1] + bins[1:]) / 2
    bins[0] -= 1 # pd.cut doesn't include left boundary value except add right=False
    G = np.array(pd.cut(X, bins, labels=[x for x in range(M)]))
    print(bins)
    print(mid_bins)
    new_X = np.array([mid_bins[x] for x in G])
    return new_X

In [29]:
X.copy()

array([0, 0, 0, 0, 1, 1, 1, 0, 0, 0, 0, 1, 1, 1, 0, 1, 0, 0, 1, 1, 0, 1,
       0, 1, 1, 1, 1, 0, 1, 1, 1, 0, 1, 0, 1, 0, 1, 0, 0, 0, 1, 0, 0, 0,
       1, 0, 1, 0, 0, 1, 0, 1, 1, 0, 0, 0, 0, 1, 0, 1, 1, 0, 1, 0, 0, 0,
       1, 0, 1, 0, 1, 1, 1, 0, 0, 1, 1, 0, 0, 0, 1, 0, 0, 0, 0, 1, 0, 1,
       1, 0, 1, 1, 0, 0, 0, 1, 1, 0, 0, 0])